This is a general exploration of zillow's dataset of home prices from April 1996 to August 2016. 
Source: https://www.zillow.com/research/data/

In [16]:
import numpy as np
import pandas as pd


import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import math

In [17]:
home_prices = pd.read_csv("City_Zhvi_AllHomes.csv")
home_prices.head(5)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


I would like to visualize the home prices of any city. Some cities have missing price values in the oldest years, so I would also like to start graphing at the beginning of actual numerical values. I will add on the start date of the data as another column onto the table, and then splice the year and price arrays to the correct length. 

In [18]:
def find_start_year_of_data(tbl):
    """Return array of the start year of the data, in string format"""
    column_names = list(home_prices)
    only_year_column_names = column_names[6:]  #there are six columns that carry nonprice information
    
    start_year_list = []
    for i in range(len(home_prices.index)):
        row = np.array(home_prices.iloc[i])

        number_missing_prices = 0
        for price in row[6:]:
            if math.isnan(price):
                number_missing_prices += 1
                
        start_year = only_year_column_names[number_missing_prices] 
        start_year_list.append(start_year)
    return start_year_list

home_prices["Start Year of Data"] = np.array(find_start_year_of_data(home_prices))
home_prices.head(5)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,Start Year of Data
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,576200,578400,582200,588000,592200,592500,590200,588000,586400,2004-08
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,560800,562800,565600,569700,574000,577800,580600,583000,585100,1996-04
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,206900,206200,205800,206200,207300,208200,209100,211000,213000,1996-04
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,121600,121800,123300,125200,126400,127000,127400,128300,129100,1996-04
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,185300,186600,188000,189100,190200,191300,192800,194500,195900,1996-04


For ease of access, I'm going to make a dictionary of the city and its row index in the table. 

In [23]:
def item_index_dict(tbl, column_name):
    """This cell makes a dictionary of a row item and its index"""
    dict_item_and_index = {}
    row_index = 0

    for item in tbl[column_name]:
        dict_item_and_index[item] = row_index
        row_index += 1
        
    return dict_item_and_index

dict_city_rowindex = item_index_dict(home_prices, "RegionName")

I'll also write functions that return tuples of only numerical price values and the start date of data for a city.

In [24]:
def get_prices_and_start_date(city):
    """This function cleans the data. 
    
    Takes in a city name.   
    Returns a tuple of actual prices and start date of data for a city.
    """
    index = dict_city_rowindex[city]
    city_row_array = np.array(home_prices.iloc[index])
    
    
    column_names = list(home_prices)
    start_date = city_row_array[-1]
    start_date_idx = column_names.index(start_date)  
    city_row_only_prices = city_row_array[start_date_idx:-1]
    
    
    return (city_row_only_prices, start_date) 

Now I can plot the appreciation over time.

In [21]:
# Expected format for string is "YYYY-MM"
def convert_to_decimal_year(string):
    year, month = string.split("-")
    return round(int(year) + int(month)/12, 2)

In [22]:
#Plots the appreciation of houses over time for a given city
def plot_appreciation(city, tbl):
    
    prices, start_date = get_prices_and_start_date(city)
    city_home_prices_float = np.array([float(price) for price in prices])
    
    column_names = list(home_prices)
    start_date_idx = column_names.index(start_date)
    
    years_string = column_names[start_date_idx:-1]
    years_int = np.array([convert_to_decimal_year(x) for x in years_string])
    
    df = pd.DataFrame({'Year': years_int, 'Home Price': city_home_prices_float})
    df.plot.line(x='Year', y='Home Price')
   
    
    
plot_appreciation("New York", home_prices)

NameError: name 'dictionary_city_rowindex' is not defined

I'd like to see what cities have appreciated the most from 1996 to 2016. Some of the cities will not show up because they are lacking a 1996 value, but that is ok for now. 

In [ ]:
inital_prices = home_prices["1996-04"]
end_prices = home_prices["2016-08"]

appreciation_rate = ((end_prices - inital_prices)/inital_prices) * 100

home_prices["Appreciation (%)"] = appreciation_rate
sorted_appreciation = home_prices.sort_values(by="Appreciation (%)", ascending=False)
sorted_appreciation.head(10)

Here we can see that out of the top ten results of the highest appreciation rate, nine are from the state of California. Is this because homes in California have actually appreciated the most of anywhere else in the United States? Or is this because the data itself is biased, and many states lack data until much later years? For example, I know from personal experience that New York City has appreciated a lot over the past 20 years. However, it will not show up in the above table because it is missing data until 2004. It's appreciation only show up as "Nan" value.

Let us look at the completeness of data by state. If California is near the top of the list, it could be that California data is the most complete and that our appreciation calculationsmay just be being influenced by that.

In [ ]:
complete_years_data = []

end = convert_to_decimal_year("2016-08")

for start_year in home_prices["Start Year of Data"]:
    start = convert_to_decimal_year(start_year)
    years_of_data = end - start
    complete_years_data.append(years_of_data)
    
home_prices["Years of Complete Data"] = complete_years_data


mean_completeness = home_prices[["State", "Years of Complete Data"]].groupby("State").mean()
mean_completeness.sort_values(by="Years of Complete Data", ascending=False)

I will calculate the median number of years of data as a frame of reference of where California is compared to the rest of the US.

In [ ]:
np.median(mean_completeness["Years of Complete Data"])

I was also curious about how many of the top 30 appreciation values belong to the state of California,

In [ ]:
first_thirty = sorted_appreciation[:30]
num_CA = len(first_thirty[(first_thirty['State']=="CA")].index)   
num_CA

California has about 19.5 years of data per city, which compares with an median of 18.5 years of data for the united states. Out of all the states, California has the 12th highest number of complete values. We can say that California does have more complete datasets than other states, and this is a possible contributing factor to California's high appreciation rate. However, other states that rank similarly to california in terms of missingness do not show up in the top appreciation rates. In fact, out of the top thirty appreciations, 27 are from California. It is highly unlikey that this occured by chance and is more likely to have arisen from other factors. Such factors could be Prop 13, a short supply of housing, favorable weather, and high paying employers in the entertainment and tech industries that drive up home prices in California. 